In [2]:
import numpy as np
from astropy.io import fits

### We Standardize all spectra

In [3]:
archivos= ['DataDESI_1_76.fits', 'DataDESI_77_152.fits', 'DataDESI_153_213.fits', 'DataDESI_214_284.fits', 'DataDESI_285_351.fits', 'DataDESI_352_438.fits'
             , 'DataDESI_439_530.fits', 'DataDESI_531_606.fits', 'DataDESI_607_690.fits', 'DataDESI_691_752.fits']

In [3]:
B_mean=[]# en estas listas se guardaran todos los espectros
Z_mean=[]
R_mean=[]
B_var = []  # Lista para guardar las varianzas
Z_var = []
R_var = []
N_espectros= 0 #Este se utiliza para calcular la media de cada pixel
N_espectros_menos_1= 0 #Este se utiliza para calcular la varianza agrupada 

n=0

for h in range(len(archivos)):
    espc = fits.open(archivos[h]) #open file
    len_espc= len(espc[2].data)
    
    #leemos la informacion
    Bflux=espc[2].data
    Rflux=espc[3].data
    Zflux=espc[4].data

    promedios_b = np.mean(Bflux, axis=0)
    promedios_r = np.mean(Rflux, axis=0)
    promedios_z = np.mean(Zflux, axis=0)

    promedios_b_promedio = [arr * (len_espc) for arr in promedios_b] #Estas guardan NhPh
    promedios_r_promedio = [arr * (len_espc) for arr in promedios_r]
    promedios_z_promedio = [arr * (len_espc) for arr in promedios_z]

    N_espectros+= len_espc #Va sumando los espectros para tener en cuenta la muestra total

    B_mean.append(promedios_b_promedio) #Estas guardan N1P1,N2P2,...,N11P11
    Z_mean.append(promedios_z_promedio)
    R_mean.append(promedios_r_promedio)


    varianza_b = np.var(Bflux, axis=0)
    varianza_r = np.var(Rflux, axis=0)
    varianza_z = np.var(Zflux, axis=0)

    varianza_b_varianza = [arr * (len_espc-1) for arr in varianza_b] #Estas guardan (Nh-1)sigmah
    varianza_r_varianza = [arr * (len_espc-1) for arr in varianza_r]
    varianza_z_varianza = [arr * (len_espc-1) for arr in varianza_z]

    N_espectros_menos_1+= len_espc-1

    B_var.append(varianza_b_varianza) #Estas guardan (N1-1)sigma1,(N2-1)sigma2,...,(N11-1)sigma11
    Z_var.append(varianza_z_varianza)
    R_var.append(varianza_r_varianza)
    
    n+=1
    print(n)


ponderacion_B = [sum(valores) for valores in zip(*B_mean)] #Suma N1P1+N2P2+...+N11P11 solo falta dividir entre la cantidad total
ponderacion_Z = [sum(valores) for valores in zip(*Z_mean)]
ponderacion_R = [sum(valores) for valores in zip(*R_mean)]

media_pixel_B = np.array([elemento / N_espectros for elemento in ponderacion_B])#Estas son las listas con los valores promedios
media_pixel_Z = np.array([elemento / N_espectros for elemento in ponderacion_Z])# en cada pixel
media_pixel_R = np.array([elemento / N_espectros for elemento in ponderacion_R])


var_agrupada_B = [sum(valores) for valores in zip(*B_var)] #Suma (N1-1)sigma1+(N2-1)sigma2+...+(N11-1)sigma11
var_agrupada_Z = [sum(valores) for valores in zip(*Z_var)]
var_agrupada_R = [sum(valores) for valores in zip(*R_var)]

var_B = np.array([elemento / N_espectros_menos_1 for elemento in var_agrupada_B])#Estas son las listas con las varianzas
var_Z = np.array([elemento / N_espectros_menos_1 for elemento in var_agrupada_Z])# en cada pixel
var_R = np.array([elemento / N_espectros_menos_1 for elemento in var_agrupada_R])


desv_B = np.sqrt(var_B) #Estas son las listas con las desviaciones en cada pixel
desv_Z = np.sqrt(var_Z)
desv_R = np.sqrt(var_R)

1
2
3
4
5
6
7
8
9
10


### We use this mean and desv values to create a new set of standardized spectra

In [4]:
#archivos= ['DataDESI_1_76.fits', 'DataDESI_77_152.fits', 'DataDESI_153_213.fits', 'DataDESI_214_284.fits', 'DataDESI_285_351.fits', 'DataDESI_352_438.fits'
#             , 'DataDESI_439_530.fits', 'DataDESI_531_606.fits', 'DataDESI_607_690.fits', 'DataDESI_691_752.fits']
archivo= ['DataDESI_691_752.fits']

In [5]:
for h in range(len(archivo)):
    espc = fits.open(archivo[h])
    
    B_FLUX_STAN=[]
    Z_FLUX_STAN=[]
    R_FLUX_STAN=[]
    
    for i in range(len(espc[4].data)):
        espc_b=((espc[2].data[i])-media_pixel_B)/desv_B
        espc_z=((espc[4].data[i])-media_pixel_Z)/desv_Z
        espc_r=((espc[3].data[i])-media_pixel_R)/desv_R
    
        B_FLUX_STAN.append(espc_b)
        Z_FLUX_STAN.append(espc_z)
        R_FLUX_STAN.append(espc_r)
#        print("copiado_1")
    
    B_FLUX_STAN=np.array(B_FLUX_STAN, dtype=np.float32)
    Z_FLUX_STAN=np.array(Z_FLUX_STAN, dtype=np.float32)
    R_FLUX_STAN=np.array(R_FLUX_STAN, dtype=np.float32)
    
    
    ##### AQUI DEBEMOS CREAR UN ARCHIVO.FITS EN DONDE VUELVA A GUARDAR LA MISMA TABLA DE LA OTRA VEZ Y AHORA B_FLUX_STAN, Z_FLUX_STAN, R_FLUX_STAN.
    # Crea un objeto ImageHDU
    hdub = fits.ImageHDU(B_FLUX_STAN)
    hduz = fits.ImageHDU(Z_FLUX_STAN)
    hdur = fits.ImageHDU(R_FLUX_STAN)
    
    # Establece el nombre del HDU como 'B_FLUX'
    hdub.name = 'B_FLUX_S'
    hdur.name = 'R_FLUX_S'
    hduz.name = 'Z_FLUX_S'
    
    #Agrega al fits.
    nombre_archivo = 'DataDESI.fits'
    hdulist = fits.open(nombre_archivo, mode='append')
    bintable_hdu = fits.BinTableHDU(espc[1].data)
    bintable_hdu.header = espc[1].header
    hdulist.append(bintable_hdu)
    hdulist.append(hdub)
    hdulist.append(hdur)
    hdulist.append(hduz)
    print("Copiado")
    hdulist.close() 

Copiado


In [6]:
from astropy.table import Table
espc = fits.open('DataDESI.fits')
espc.info()
print(len(espc[2].data))

Filename: DataDESI.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     26   89878R x 9C   [7A, D, D, 7A, D, 21A, D, D, 7A]   
  2  B_FLUX_S      1 ImageHDU         8   (2751, 89878)   float32   
  3  R_FLUX_S      1 ImageHDU         8   (2326, 89878)   float32   
  4  Z_FLUX_S      1 ImageHDU         8   (2881, 89878)   float32   
89878


In [7]:
Table.read(espc, hdu=1) 

SPECTYPE,Z,TARGETID,OBJTYPE,Z_ERR,SUBTYPE,TARGET_RA,TARGET_DEC,MORPHTYPE
str7,float64,float64,str7,float64,str21,float64,float64,str7
GALAXY,0.12304444393698347,3.963298676893649e+16,TGT,4.1065983890874034e-05,0.0,251.15602592812982,35.15999350466772,SER
GALAXY,0.29457611574915027,3.963298676893617e+16,TGT,1.5230988695018649e-05,0.0,251.13617942652738,35.298171294155715,REX
GALAXY,0.26295948028332505,3.963298677312854e+16,TGT,1.8882726599351002e-05,0.0,251.29263317255828,35.2563912473969,REX
GALAXY,1.1613819989817356,6.160937390723566e+17,SKY,9.201895922269369e-05,0.0,251.18910901219022,35.19275651875039,0.0
GALAXY,0.26355166471566344,3.963298677312746e+16,TGT,3.503667072881993e-05,0.0,251.22081435317995,35.196753502348336,EXP
GALAXY,0.08709363315046506,3.963298677312904e+16,TGT,3.865175488516558e-06,0.0,251.32575334761174,35.28205469932947,EXP
GALAXY,0.26495553753651446,3.9632986768936136e+16,TGT,7.675386768239847e-05,0.0,251.13412200880515,35.23854486920474,SER
STAR,-0.00017170741474576307,3.963298676474267e+16,TGT,6.297295578494421e-06,G,250.85477816221342,35.164102405409004,PSF
GALAXY,0.1138600133189154,3.963298676893551e+16,TGT,7.161186384705142e-05,0.0,251.09286363782294,35.15489017306431,SER


In [8]:
espc[2].data

array([[-2.0244894 , -1.2554272 ,  0.14935206, ...,  0.02944778,
        -0.06924197, -0.07051332],
       [-0.38240528, -0.843131  ,  0.1980905 , ..., -0.4721748 ,
        -0.34204477, -0.10186274],
       [-2.0898461 , -0.94940627, -0.8364523 , ..., -0.18528688,
        -0.06044401, -0.05745404],
       ...,
       [-0.26975256,  0.02906103,  0.3725471 , ..., -0.29895002,
        -0.22854656, -0.21899547],
       [-0.25106573, -0.16347565,  0.00787027, ..., -0.3659452 ,
        -0.3861323 , -0.20527843],
       [-0.64394575,  0.2515735 , -0.23941648, ..., -0.29345423,
        -0.3058742 , -0.33522555]], dtype=float32)

In [9]:
espc.close()